In [ ]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
import json

config = json.load((Path.cwd().parent / 'config.json').open('rb'))
network_fp = Path(config['project_directory']) / "Network"
osmdwnld_fp = Path(config['project_directory']) / "OSM_Download"
elevation_fp = Path(config['project_directory']) / "Elevation"
cyclinginfra_fp = Path(config['project_directory']) / "Cycling_Infra_Dating"
calibration_fp = Path(config['project_directory']) / "Calibration"


In [ ]:
import requests

state_two_letter = "ga"
YYYY = 2010

lodes_url_base = f"https://lehd.ces.census.gov/data/lodes/LODES7/{state_two_letter}/od/{state_two_letter}_od_main_JT00_{YYYY}.csv.gz"

In [ ]:
import pandas as pd
test = pd.read_csv(lodes_url_base)


In [ ]:
rename_dict = {
    'w_geocode':'Work Block ID',
    'h_geocode':'Home Block ID',
    'S000': 'Total Jobs'  
}
test.rename(columns=rename_dict, inplace=True)
test = test[['Work Block ID','Home Block ID','Total Jobs']]

In [ ]:
import geopandas as gpd
studyarea = gpd.read_file("/Users/tannerpassmore/Documents/BikewaySim/RAW/SAGIS/savannah_metro.geojson")#.to_crs(config['projected_crs_epsg'])
studyarea = studyarea.unary_union.envelope

In [ ]:
blocks = gpd.read_file("/Users/tannerpassmore/Documents/BikewaySim/Savannah/OD Creation/tl_2010_13_tabblock10.shp",mask=studyarea).to_crs(config['projected_crs_epsg'])

In [ ]:
#turn to int
test['Work Block ID'] = test['Work Block ID'].astype(int)
test['Home Block ID'] = test['Home Block ID'].astype(int)
blocks['GEOID10'] = blocks['GEOID10'].astype(int)

In [ ]:
works = test[['Work Block ID','Total Jobs']].copy()
works = works.groupby('Work Block ID')['Total Jobs'].sum().reset_index()
works = pd.merge(works,blocks[['GEOID10','geometry']],left_on="Work Block ID",right_on='GEOID10')
works = gpd.GeoDataFrame(works,geometry='geometry')

In [ ]:
works.explore('Total Jobs')

In [ ]:
merged = pd.merge(test,blocks[['GEOID10','geometry']],left_on="Work Block ID",right_on='GEOID10',how='left')
merged.drop(columns=['GEOID10'],inplace=True)
merged.rename(columns={'geometry':'work_geo'},inplace=True)
merged = pd.merge(merged,blocks[['GEOID10','geometry']],left_on="Home Block ID",right_on='GEOID10',how='left')
merged.drop(columns=['GEOID10'],inplace=True)
merged.rename(columns={'geometry':'home_geo'},inplace=True)
merged = merged[merged['home_geo'].notna() & merged['work_geo'].notna()]

In [ ]:
random_id = merged['Work Block ID'].sample(1).item()
subset = merged.loc[merged['Work Block ID']==random_id].copy()
subset = gpd.GeoDataFrame(subset,geometry='home_geo')
m = subset.explore()

In [ ]:
blocks.loc[blocks['GEOID10']==random_id,'geometry'].explore()

In [ ]:
work = blocks.loc[blocks['GEOID10']==random_id,'geometry'].to_crs('epsg:4326').item().centroid

In [ ]:
x, y = list(work.coords)[0]

In [ ]:
import folium
folium.Circle((y,x),radius=100,tooltip='WORK',kwargs={'color':'green'}).add_to(m)
m

In [ ]:
merged['work_geo'] = gpd.GeoSeries(merged['work_geo']).centroid
merged['home_geo'] = gpd.GeoSeries(merged['home_geo']).centroid

In [ ]:
# import geopandas as gpd
# import pandas as pd
# from pathlib import Path
# import json

# config = json.load((Path.cwd().parent / 'config.json').open('rb'))
# network_fp = Path(config['project_directory']) / "Network"
# osmdwnld_fp = Path(config['project_directory']) / "OSM_Download"
# elevation_fp = Path(config['project_directory']) / "Elevation"
# cyclinginfra_fp = Path(config['project_directory']) / "Cycling_Infra_Dating"
# calibration_fp = Path(config['project_directory']) / "Calibration"

In [ ]:



#'foot' in blocks.crs.axis_info[0].unit_name

#df.crs.axis_info[0].unit_name